In [63]:
# %pip install pandas
# %pip install openpyxl
# %pip install datamatch

In [64]:
import pandas as pd 
from datamatch import JaroWinklerSimilarity, ThresholdMatcher, ColumnsIndex, DateSimilarity


In [65]:
def read():
    dfa = pd.read_csv("../../../data/LA/IPNO/allegation.csv")
    dfb = pd.read_csv("../../../data/LA/IPNO/personnel.csv")

    df = pd.merge(dfa, dfb, on="uid")
    return df

df = read()
df

per = df[["first_name", "middle_name", "last_name", "sex_y", "race_y", "agency_y", "allegation", "allegation_uid", "action", "allegation_desc", "uid"]]

per = per.rename(columns={"sex_y": "sex", "race_y": "race", "agency_y": "agency"})

per.loc[:, "first_name"] = per.first_name.str.lower().str.strip()
per.loc[:, "middle_name"] = per.middle_name.str.lower().str.strip()
per.loc[:, "last_name"] = per.last_name.str.lower().str.strip()

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_3624/2367025626.py:2: DtypeWarning: Columns (3,10,14,15,16,18,19,20,22,25,26,27,28,31,32,33,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../../../data/LA/IPNO/allegation.csv")


In [66]:
df.columns

Index(['uid', 'tracking_id_og', 'tracking_id', 'case_number', 'allegation_uid',
       'allegation', 'investigation_status', 'assigned_department',
       'assigned_division', 'traffic_stop', 'body_worn_camera_available',
       'app_used', 'citizen_arrested', 'disposition', 'complainant_name',
       'action', 'initial_action', 'agency_x', 'incident_type',
       'supervisor_uid', 'supervisor_rank', 'badge_no', 'department_code',
       'department_desc', 'employment_status', 'investigator',
       'investigator_uid', 'investigator_rank', 'shift_supervisor',
       'allegation_desc', 'investigating_department', 'referred_by',
       'incident_location', 'disposition_desc', 'item_number', 'race_x',
       'sex_x', 'age', 'arresting_officer_uid', 'investigating_supervisor_uid',
       'reporting_officer_uid', 'coaccusal', 'last_name', 'middle_name',
       'first_name', 'birth_year', 'birth_month', 'birth_day', 'race_y',
       'sex_y', 'agency_y'],
      dtype='object')

In [67]:
# per.to_csv("../../../data/LA/IPNO/llead.csv", index=False)

In [68]:
def read_data():
    la = pd.read_csv("../../../data/LA/IPNO/llead.csv")
    tx = pd.read_csv("../../../data/TX/clean/tx_reciprocity-2023.csv")

    return la, tx

In [69]:
la, tx = read_data()

In [70]:
from uid import gen_uid

tx = tx.pipe(gen_uid, ["first_name", "last_name"])

In [71]:
tx

,Course,first_name,last_name,DATE_CERTIFIED,location,uid
0,Out of State / Federal,edward,use,2020-08-04,louisiana,6aa2a95c78b09b57abdc4b387300c749
1,Out of State / Federal,terrill,vandergriff,2017-05-17,louisiana,e0538da0e397ac24a0b67f763c19ce02
2,Out of State / Federal,michael,mims,2023-04-03,louisiana,f33bd010fcc6a507bc19b805a95c9656
3,Out of State / Federal,dirk,chapman,2021-06-28,louisiana,26d95c2d0107359ef07bf7b3bab28796
4,Out of State / Federal,shaun,sylvas,2023-05-18,louisiana,cbda1d6dbb611698e59a92ba6e507bd4
...,...,...,...,...,...,...
60,Out of State / Federal,lucas,hvasta,2023-04-17,louisiana,f1a1fb2e59fc282725a4b68c189b41ec
61,Out of State / Federal,john,taylor,2023-03-13,louisiana,d65bac66538166828f57c8c499453dbc
62,Out of State / Federal,steven,griego,2023-02-03,louisiana,0adb68a1c328696768840923a807f061
63,Out of State / Federal,joseph,hawkins,2022-12-30,louisiana,a4b43f5abb849cb1b73bfba341e217aa


In [72]:
la

,first_name,middle_name,last_name,sex,race,agency,allegation,allegation_uid,action,allegation_desc,uid
0,marcus,NaN,aucoin,NaN,NaN,13th-da,in service deficiency,52a76c208116feb297addde4756f38c5,decertified,NaN,62a0c238a91b9c0c3e7591ddd1dffd9c
1,stanley,NaN,rabalais,NaN,NaN,22nd-da,in service deficiency,574310e1c270d70ae4d957c19f2b7227,decertified,NaN,0b6f832a168ce3ef8b822467ddffb862
2,stephen,NaN,lambert,NaN,NaN,25th-judicial-district-court,in service deficiency,31353e37c04217efeec75e9a5b61fc77,decertified,NaN,fedd1368fd9bafdc965b3751d877a99a
3,columbus,NaN,willis,NaN,NaN,6th-judicial-district-court,in service deficiency,468053d0167e37e0c791c3c41f77f351,decertified,NaN,1b4b2e05daf9d10ba48023c125ba6e6e
4,billy,NaN,zeigler,NaN,NaN,6th-judicial-district-court,in service deficiency,f03f0ab6f6db30f51339fb5eae0e9a03,decertified,NaN,198affae289d702df23977757228bb04
...,...,...,...,...,...,...,...,...,...,...,...
39252,kevin,NaN,stimage,NaN,NaN,tangipahoa-so,theft,ea4b28baad94ab532b14db5dd4af4d11,NaN,NaN,6c341196fc8cea90dc56fd4b97f99107
39253,darryl,b,stewart,NaN,NaN,terrebonne-so,theft,55e3960883da91edc488558cf6c82690,NaN,NaN,2a43d64a898c219e4f3a27484b13882d
39254,dawn,c,foret,NaN,NaN,terrebonne-so,theft,a5374061b7204bf723b1d66834a7d886,NaN,NaN,9790fc4799e09722d14543ad21c10c5d
39255,nathaniel,NaN,savoy,NaN,NaN,ville-platte-pd,dishonesty,b9654915d78fb7e7a9d93087dd3dd88e,NaN,NaN,e4df26cee92d192d9a4adca5e69ef794


In [73]:
la.loc[:, "full_name"] = la.first_name.str.cat(la.last_name, sep=" ")

In [74]:

def match_la_with_tx_reciprocity(la, tx):
    dfa = la
    dfa["state"] = "Louisiana"
    dfa = (
        la.loc[la.uid.notna(), ["uid", "first_name", "last_name", "allegation", "state"]]
        .drop_duplicates(subset=["uid"])
        .set_index("uid", drop=True)
    )
    dfa.loc[:, "fc"] = dfa.first_name.fillna("").map(lambda x: x[:1])


    dfb = tx
    dfb["allegation"] = ""
    # dfb["action"] = ""
    # dfb["allegation_uid"] = ""
    # dfb["allegation_desc"] = ""
    # dfb["full_name"] = ""
    dfb["state"]  = "Texas"

    dfb = (
        tx.loc[tx.uid.notna(), ["uid", "first_name", "last_name", "allegation", "state"]]
        .drop_duplicates(subset=["uid"])
        .set_index("uid", drop=True)
    )
    dfb.loc[:, "fc"] = dfb.first_name.fillna("").map(lambda x: x[:1])

    matcher = ThresholdMatcher(
        ColumnsIndex(["fc"]),
        {
            "first_name": JaroWinklerSimilarity(),
            "last_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.958
    matcher.save_pairs_to_excel(
        ("../../../data/TX/match/tx_reciprocity-2023.xlsx"),
        decision,
    )
    matches = matcher.get_index_pairs_within_thresholds(decision)
    match_dict = dict(matches)

    la.loc[:, "uid"] = tx.uid.map(lambda x: match_dict.get(x, x))
    return tx

match_data = match_la_with_tx_reciprocity(la, tx)

In [ ]:

# la_matches = la[la.full_name.isin(["ronald moore", "bryan jackson", "jake engle", 
#                                    "james winston", "janavian chocklin", "jeffrey allday", 
#                                    "taylor giroir", "cody parker", "christopher davis",
#                                    "christian hicks", "kregg jones", "kenneth thompson", "kwouska lee",
#                                    "karl ratcliff", "lucas yocum", "lucas hvasta", "matthew tippett", "michael mims"])]
# la_matches.to_csv("../../../data/TX/match/matches.csv")